In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;

In [2]:
# Max Problem (to obtain the values of x and z given an interdiction)

def InnerProblem (gamma):
    
    max_model = gp.Model("Max_z");
    
    x = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = max_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sum_z = max_model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            
    max_model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                         - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    for i,j in G.edges:
        max_model.addConstr(x[i,j] - (1-gamma[i,j].x)*G.edges[i,j]['capacity'] <= 0);
            
    for i,j in G.edges:
        max_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
            
    max_model.addConstr(alpha[t] - alpha[s] >= 1);
            
    max_model.addConstr(x[t,s] - gp.quicksum((1-gamma[i,j].x)*G.edges[i,j]['capacity']*theta[i,j]
                                                     for i,j in G.edges) >= 0);
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            max_model.addConstr(x[i,j] - (1/M)*z[i,j] >= 0);
            
    max_model.addConstr(sum_z <= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
    max_model.setObjective(sum_z, GRB.MAXIMIZE); 
            
    #max_model.setParam("IntegralityFocus",1);
    #max_model.setParam("NumericFocus",2);
            
    #max_model.setParam('TimeLimit', T_Lim);
            
    max_model.update();
    max_model.setParam("OutputFlag", 0);
    max_model.optimize();
      
    return x, z;


In [3]:
#'''
# Creating Results Files ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

file_summary = open("RLP_Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, Cal_A_Level\t, Nodes\t, Arcs\t, OptGap\t,');
file_summary.write('Obj_value\t, Flow\t, Level_1\t, Level_2\t, Level_3\t, Other_Levels\t, Run_Time\n');
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#'''

num = 10;

model = {};
budget = [0, 5, 10, 15];
rate = [1];

T_Lim = 60;

network = {};


# Input -- Reading the networks
for n in range(num):
    model[n] = {};
    network[n] = 'Net'+str(n+1);
     
    for b in range(len(budget)):  
        model[n][b] = {};
        
        for r in range(len(rate)):
                        
            start_time = time.time(); 
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
                calA_level = int(row1[2]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);                        
                                           
                    
                A = 0;
                U = G.edges[t,s]['capacity'];

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;                     
            
            M = A;
            Mo = A;
            Me = 1e6;


            model[n][b][r] = gp.Model("Pessimistic_Relaxed_LP"); 
            
            gamma = model[n][b][r].addVars(G.edges, vtype=GRB.BINARY);
            chi = model[n][b][r].addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY);
            omega = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            phi = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            eta = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            kappa = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            delta = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            epsilon = model[n][b][r].addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            pi = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            nu = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            psi = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            xi = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
            
            val = model[n][b][r].addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY);
            
            x = model[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            z = model[n][b][r].addVars(G.edges, vtype=GRB.BINARY);
            
                
            
            # Constraints
            model[n][b][r].addConstr(val >= gp.quicksum(psi[i,j]*G.edges[i,j]['capacity'] for i,j in G.edges) +
                                     pi + gp.quicksum(epsilon[i] for i in G.nodes) + 
                                     gp.quicksum(nu[i,j] for i,j in G.edges) + 
                                     gp.quicksum(kappa[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
            # Upper-Level
            model[n][b][r].addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget[b]);
            
            
            model[n][b][r].addConstr(chi[t] - chi[s] + omega[t,s] + eta >= 0);
            
            for i,j in G.edges:
                if (G.edges[i,j]['special'] == 1):
                    model[n][b][r].addConstr(chi[i] - chi[j] + omega[i,j] + phi[i,j] >= 0);
                elif (i != t and j!= s):
                    model[n][b][r].addConstr(chi[i] - chi[j] + omega[i,j] >= 0);
            
            for i,j in G.edges:
                if (G.edges[i,j]['special'] == 1):
                    model[n][b][r].addConstr(kappa[i,j] - (1/M)*phi[i,j] >= 1);
                       
            
            model[n][b][r].addConstr(gp.quicksum(delta[t,j] for j in G.successors(t))
                 - gp.quicksum(delta[j,t] for j in G.predecessors(t)) + epsilon[t] + pi >= 0);
            
            model[n][b][r].addConstr(gp.quicksum(delta[j,s] for j in G.predecessors(s))
                 - gp.quicksum(delta[s,j] for j in G.successors(s)) + epsilon[s] - pi >= 0);
            
            for i in G.nodes:
                if i != s and i != t:
                    model[n][b][r].addConstr(gp.quicksum(delta[i,j] for j in G.successors(i))
                                             - gp.quicksum(delta[j,i] for j in G.predecessors(i)) + epsilon[i] >= 0);
                    
            for i,j in G.edges:
                model[n][b][r].addConstr(delta[i,j] - G.edges[i,j]['capacity']*xi[i,j] + nu[i,j] >= 0);
                model[n][b][r].addConstr(psi[i,j] <= omega[i,j]);
                model[n][b][r].addConstr(psi[i,j] <= (1-gamma[i,j])*Mo);
                model[n][b][r].addConstr(psi[i,j] >= omega[i,j] - Mo*gamma[i,j]);
                model[n][b][r].addConstr(xi[i,j] >= eta);
                model[n][b][r].addConstr(xi[i,j] >= -(1-gamma[i,j])*Me);
                model[n][b][r].addConstr(xi[i,j] <= eta + gamma[i,j]*Me);
                    
                    
            model[n][b][r].setObjective(val, GRB.MINIMIZE); 
            
            model[n][b][r].setParam("IntegralityFocus",1);
            #model[n][b][r].setParam("NumericFocus",2);
            
            model[n][b][r].setParam('TimeLimit', T_Lim);
            
            model[n][b][r].update();
            model[n][b][r].setParam("OutputFlag", 0);
            model[n][b][r].optimize();
            
            
            # Solving Inner Problem to obtain x* and z*
            
            InnerProblem (gamma);
            

                    
            
            
            # ~~~~~~~~~~~~~~~ Printing Results ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            print('\n');
            print(network[n], ', budget', budget[b]);
            
            file = open('RLP_results_'+network[n]+'_b'+str(budget[b])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (t+1) +'\n');
            file.write('Number of Arcs: %g' % G.number_of_edges() +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            if model[n][b][r].status == GRB.status.OPTIMAL:
                obj = model[n][b][r].objVal;
                OptGap = 0.0;
                sol = A - obj;
                
                print('Solved to Optimality');
                print('Number of special arcs without flow: %g' % sol);
                print('Number of special arcs with flow: %g' % obj);
                print('\n');
                
                file.write('Solved to Optimality\n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
                
                
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0:
                        print('gamma: ',i,j, gamma[i,j].x);
                        
                
            elif model[n][b][r].status == GRB.status.TIME_LIMIT:
                obj = model[n][b][r].objVal;
                LB = model[n][b][r].objBound;
                sol = A - obj;
                OptGap = model[n][b][r].MIPGap;
                
                '''
                if obj != 0.0:
                    OptGap = abs((obj-LB)/obj);
                else:
                    OptGap = abs(obj-LB);
                '''
                    
                print('Reached Time Limit');
                print('Number of special arcs without flow: %g' % sol);
                print('Number of special arcs with flow: %g' % obj);     
                print('Optimality Gap: %g' % OptGap);
                print('\n');
                
                for i,j in G.edges:
                    if gamma[i,j].x > 0.0:
                        print('gamma: ',i,j, gamma[i,j].x);
                
                
                file.write('Reached Time Limit \n');
                file.write('Number of special arcs without flow: %g' % sol +'\n');
                file.write('Number of special arcs with flow: %g' % obj +'\n');
                file.write('UB: %g' % obj +'\n'); 
                file.write('LB: %g' % LB +'\n');
                file.write('Opt Gap: %s' % str("{:.2f}".format((OptGap)))+'\n');
            
            
            file.write('\nMax-Flow: %g' % x[t,s].x +'\n');
            file.write('Run-time: %s' % "{:.2f}".format((time.time() - start_time)) + ' sec');
            file.write('\n\n');    
            file.write('Level 0 Capacities: ' + str(G.edges[s,1]['capacity']) +'\n\n');
            
            lev1 = 0;
            lev2 = 0;
            lev3 = 0;
            other_level = 0;
            file.write('\nInterdiction plan: \n');
            
            key1 = False;
            key2 = False;
            key3 = False;
                
            for i, j in G.edges: 
                if gamma[i,j].x > 0.0000001:
                    if G.edges[i,j]["level"] == 1:
                        if key1 == False:
                            file.write("Level 1:" +'\n');
                            key1 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev1 += 1;
                    elif G.edges[i,j]["level"] == 2:
                        if key2 == False:
                            file.write('Level 2:' +'\n');
                            key2 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev2 += 1;
                    elif G.edges[i,j]["level"] == 3:
                        if key3 == False:
                            file.write('Level 3:' +'\n');
                            key3 = True;
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        lev3 += 1;
                    else:
                        file.write('Other Levels:' +'\n');
                        file.write('Arc (%s,%s)' %(i,j) +'\n');
                        other_level += 1;
                   
                        
            
            file.write('\n');
            file.write('Special arcs with flow' +'\n');    
                
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if x[i,j].x > 0.0000001:
                        file.write('Arc (%s,%s): ' %(i,j) + '\t Flow: %f' % x[i,j].x +'\n');
                        
                        
            file_summary.write(network[n]+'_b'+str(budget[b])+','+str(budget[b])+','+str(calA_level)+','
                               +str(t+1)+','+str(G.number_of_edges())+','+str("{:.3f}".format((OptGap)))+','+str(obj)+','
                               +str("{:.2f}".format(x[t, s].x))+','+str(lev1)+','+str(lev2)+','+str(lev3)+','
                               +str(other_level)+','+str("{:.2f}".format((time.time() - start_time)))+'sec\n');
            
            
            
            file.write('\n');
            file.close();
                         
file_summary.close();
            
   

Academic license - for non-commercial use only - expires 2024-01-14
Using license file /home/daniel/gurobi.lic
Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net1 , budget 0
Solved to Optimality
Number of special arcs without flow: 0
Number of special arcs with flow: 17


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net1 , budget 5
Solved to Optimality
Number of special arcs without flow: 6
Number of special arcs with flow: 11


gamma:  1 10 1.0
gamma:  8 12 1.0
gamma:  8 13 1.0
gamma:  8 14 1.0
gamma:  8 15 1.0
Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net1 , budget 10
Reached Tim

   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net5 , budget 15
Reached Time Limit
Number of special arcs without flow: 13
Number of special arcs with flow: 3
Optimality Gap: 442.812


gamma:  1 10 1.0
gamma:  5 9 1.0
gamma:  6 11 1.0
gamma:  6 12 1.0
gamma:  6 14 1.0
gamma:  6 15 1.0
gamma:  7 11 1.0
gamma:  7 13 1.0
gamma:  7 14 1.0
gamma:  7 15 1.0
gamma:  8 11 1.0
gamma:  8 12 1.0
gamma:  8 13 1.0
gamma:  8 14 1.0
gamma:  8 15 1.0
Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net6 , budget 0
Solved to Optimality
Number of special arcs without flow: 0
Number of special arcs with flow: 14


Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: in



Net10 , budget 10
Reached Time Limit
Number of special arcs without flow: 10
Number of special arcs with flow: 3
Optimality Gap: 194.466


gamma:  1 6 1.0
gamma:  3 7 1.0
gamma:  3 9 1.0
gamma:  4 6 1.0
gamma:  4 9 1.0
gamma:  4 10 1.0
gamma:  8 11 1.0
gamma:  8 12 1.0
gamma:  8 14 1.0
gamma:  8 15 1.0
Changed value of parameter IntegralityFocus to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


Net10 , budget 15
Reached Time Limit
Number of special arcs without flow: 12
Number of special arcs with flow: 1
Optimality Gap: 762.896


gamma:  1 6 1.0
gamma:  1 10 1.0
gamma:  3 6 1.0
gamma:  3 7 1.0
gamma:  3 9 1.0
gamma:  4 6 1.0
gamma:  4 9 1.0
gamma:  4 10 1.0
gamma:  8 11 1.0
gamma:  8 12 1.0
gamma:  8 14 1.0
gamma:  8 15 1.0
gamma:  7 14 1.0
